## Classificação por isolamento de frequência

Neste notebook será realizado um exemplo de classificação, isolando todas as frequências estimuladas no conjunto de dados `AVI SSVEP Single Target`. 

### Passos para a realização da classificação:

1. **Carrega** o arquivo `fif` (`mne.EpochsArray`) dos dados **filtrados**;
2. **Determine o limiar** para isolar cada uma das frequências estimuladas. Por exemplo, a faixa de frequência para o estímulo de 6.5 Hz irá resultar em pontos (`PSD`) que irão variar de 6.3 à 6.7 Hz, caso o limiar seja de 0.2 Hz;
3. **Obter a "energia"** do sinal por meio do cálculo `compute_psd` para cada uma das faixas de frequência que podem ser estimuladas. Por exemplo:
    - Obtenha todas as frequências estimuladas. Ex: 6, 6.5, 7, 7.5, 8.2 e 9.3;
    - Obtenha o valor mínimo e o máximo para cada frequência utilizando limiar. Ex: (5.8, 6.2), (6.3, 6.7), ...
    - Aplique o `compute_psd` para cada tupla (min, max), por meio dos parâmetros `fmin` e `fmax` do mesmo método.
4. Com as listas de pontos isoladas e computadas (`PSD`) para cada amostra, aplique um cálculo de característica adequada. Características manuais interessantes para este exemplo podem ser `max_value`, `average` ou `median`. No fim deste passo iremos obter um **vetor de características**;
5. Por fim, realize a **classificação**, que será um **cálculo de voto** simples (maior valor é provavelmente o a frequência evocada).

A seguir, um exemplo desta classificação com os dados `single target` de `AVI dataset`:

### Imports

In [1]:
import mne
import numpy as np

### Tarefa 1
Carregar os dados do arquivo `fif` dos dados filtrados.

In [2]:
data = mne.read_epochs("../../../datasets/avi/multi/mne_data.fif")

Reading c:\Users\Maria Fernanda\Desktop\SSVEP\src\avi\multi\..\..\..\datasets\avi\multi\mne_data.fif ...
Isotrak not found
    Found the data of interest:
        t =       0.00 ...   15998.05 ms
        0 CTF compensation matrices available


C:\Users\Maria Fernanda\AppData\Local\Temp\ipykernel_9656\659670335.py:1: RuntimeWarning: This filename (../../../datasets/avi/multi/mne_data.fif) does not conform to MNE naming conventions. All epochs files should end with -epo.fif, -epo.fif.gz, _epo.fif or _epo.fif.gz
  data = mne.read_epochs("../../../datasets/avi/multi/mne_data.fif")


Not setting metadata
100 matching events found
No baseline correction applied
0 projection items activated


### Tarefa 2
Determine o limiar para isolar cada uma das frequências estimuladas.

In [3]:
threshold = 0.25

# entendimento dos dados
print(data.get_data().shape)
data

(100, 1, 8192)


Number of events,100
Events,6: 406.5: 57: 107.5: 158.2: 259.3: 5
Time range,0.000 – 15.998 s
Baseline,off


### Tarefa 3
Obter a "energia" do sinal por meio do cálculo `compute_psd` para cada uma das faixas de frequência que podem ser estimuladas.

In [4]:
targets = [float(item) for item in data.event_id.keys()]
print("Possíveis alvos:", targets)

features = list()
for i in range(len(data)):
    sample = list()
    for target in targets:
        fmin = target - threshold
        fmax = target + threshold
        test = data[i].compute_psd(method='multitaper', fmin=fmin,
                                          fmax=fmax, verbose=False)
        print("DEBUG", i, test)
        sample.append(test)
    features.append(sample)

# print(features)
# X = np.array(features)
# print("Formato dos dados calculados:", features.shape)

Possíveis alvos: [6.0, 6.5, 7.0, 7.5, 8.2, 9.3]


DEBUG 0 <Power Spectrum (from Epochs, multitaper method) | 1 epochs × 1 channels × 9 freqs, 5.8-6.2 Hz>
DEBUG 0 <Power Spectrum (from Epochs, multitaper method) | 1 epochs × 1 channels × 9 freqs, 6.2-6.8 Hz>
DEBUG 0 <Power Spectrum (from Epochs, multitaper method) | 1 epochs × 1 channels × 9 freqs, 6.8-7.2 Hz>
DEBUG 0 <Power Spectrum (from Epochs, multitaper method) | 1 epochs × 1 channels × 9 freqs, 7.2-7.8 Hz>
DEBUG 0 <Power Spectrum (from Epochs, multitaper method) | 1 epochs × 1 channels × 8 freqs, 8.0-8.4 Hz>
DEBUG 0 <Power Spectrum (from Epochs, multitaper method) | 1 epochs × 1 channels × 8 freqs, 9.1-9.5 Hz>
DEBUG 1 <Power Spectrum (from Epochs, multitaper method) | 1 epochs × 1 channels × 9 freqs, 5.8-6.2 Hz>
DEBUG 1 <Power Spectrum (from Epochs, multitaper method) | 1 epochs × 1 channels × 9 freqs, 6.2-6.8 Hz>
DEBUG 1 <Power Spectrum (from Epochs, multitaper method) | 1 epochs × 1 channels × 9 freqs, 6.8-7.2 Hz>
DEBUG 1 <Power Spectrum (from Epochs, multitaper method) | 1 epo

In [5]:
X = np.array(features)
print("Formato dos dados calculados:", X.shape)


ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 4 dimensions. The detected shape was (100, 6, 1, 1) + inhomogeneous part.

In [ ]:
# organizando os dados

X = X.reshape(X.shape[0], X.shape[1], X.shape[-1])
print("Formato padronizado dos dados com PSD calculado:", X.shape)

Formato padronizado dos dados com PSD calculado: (84, 7, 15)


### Tarefa 4
Aplicação do calculo de característica adequada.

In [ ]:
# aplicando a característica de "maior valor"
max_values = np.max(X, axis=-1)
max_values.shape

(84, 7)

### Tarefa 5
Classificação.

In [ ]:
y = np.load("datasets/avi/multi/labels_formated.npy")

# entendimento dos dados
print(y, y.shape, targets)

# classificação pelo "voto" do maior valor
i_max = max_values.argmax(axis=-1)
hits = [1 for i in range(len(i_max)) if targets[i_max[i]] == y[i]]
acc = 100 * sum(hits) / len(y)
print(f'\nPorcentagem de acerto: {acc:.2f}%')

[10.  10.  10.   6.5  6.5  6.5  6.   6.   6.   7.5  7.5  7.5  7.   7.
  7.   8.2  8.2  8.2  9.3  9.3  9.3 10.  10.  10.   6.5  6.5  6.5  6.
  6.   6.   7.5  7.5  7.5  7.   7.   7.   8.2  8.2  8.2  9.3  9.3  9.3
 10.  10.  10.   6.5  6.5  6.5  6.   6.   6.   7.5  7.5  7.5  7.   7.
  7.   8.2  8.2  8.2  9.3  9.3  9.3 10.  10.  10.   6.5  6.5  6.5  6.
  6.   6.   7.5  7.5  7.5  7.   7.   7.   8.2  8.2  8.2  9.3  9.3  9.3] (84,) [6.0, 6.5, 7.0, 7.5, 8.2, 9.3, 10.0]

Porcentagem de acerto: 85.71%
